In [1]:
import pandas as pd
import numpy as np
import time
from sklearn.pipeline import Pipeline
from sklearn.model_selection import *
from sklearn.preprocessing import*
import datetime,time
from sklearn.metrics import*
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns
import re

pd.options.display.float_format = '{:.7f}'.format

In [2]:
# app_train = pd.read_csv('C:/Shared/machine_learning/home-credit-default-risk/application_train.csv')
# app_test = pd.read_csv('C:/Shared/machine_learning/home-credit-default-risk/application_test.csv')
# final_train = pd.read_csv('C:/Shared/machine_learning/home-credit-default-risk/final_train.csv')
# final_test = pd.read_csv('C:/Shared/machine_learning/home-credit-default-risk/final_test.csv')

final_train = pd.read_csv('D:/project/homecredit/final/ETL_ALL_not_encoding_v2.csv')

In [16]:
X = final_train.drop(columns= ['Unnamed: 0'], axis = 1)
y = final_train.TARGET
X = X.replace([np.inf, -np.inf], np.nan)
X.fillna(0.0, inplace = True)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.8, random_state = 0)

In [33]:
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
import gc

def model(features, test_features, encoding = 'le', n_folds = 5):
    
    """Train and test a light gradient boosting model using
    cross validation. 
    
    Parameters
    --------
        features (pd.DataFrame): 
            dataframe of training features to use 
            for training a model. Must include the TARGET column.
        test_features (pd.DataFrame): 
            dataframe of testing features to use
            for making predictions with the model. 
        encoding (str, default = 'ohe'): 
            method for encoding categorical variables. Either 'ohe' for one-hot encoding or 'le' for integer label encoding
            n_folds (int, default = 5): number of folds to use for cross validation
        
    Return
    --------
        submission (pd.DataFrame): 
            dataframe with `SK_ID_CURR` and `TARGET` probabilities
            predicted by the model.
        feature_importances (pd.DataFrame): 
            dataframe with the feature importances from the model.
        valid_metrics (pd.DataFrame): 
            dataframe with training and validation metrics (ROC AUC) for each fold and overall.
        
    """
    
    #ID 取出
    # Extract the ids
    train_ids = features['SK_ID_CURR']
    test_ids = test_features['SK_ID_CURR']
    
    #目標取出
    # Extract the labels for training
    labels = features['TARGET']
    
    #刪除train 的目標和ID ，刪除test的ID
    # Remove the ids and target
    features = features.drop(columns = ['SK_ID_CURR', 'TARGET'])
    test_features = test_features.drop(columns = ['SK_ID_CURR', 'TARGET'])
    
    
    #分兩種方法'ohe':將train和test都做one hot建立虛擬變數(攤平)，並且根據兩者都有得col項目合併
    # One Hot Encoding
    if encoding == 'ohe':
        features = pd.get_dummies(features)
        test_features = pd.get_dummies(test_features)
        
        # Align the dataframes by the columns
        features, test_features = features.align(test_features, join = 'inner', axis = 1)
        
        # No categorical indices to record
        cat_indices = 'auto'
    
    
    
    # Integer label encoding
    elif encoding == 'le':
        
        # Create a label encoder
        label_encoder = LabelEncoder()
        
        # List for storing categorical indices
        cat_indices = []
        
        #enumerate() 將資料變成((0,apptrain.columns[0]), (1,apptrain_columns[1])....)一對一放進 i 和 col跑迴圈
        #LabelEncoder.fit_transform 將 object變數做重新編碼從 0 開始
        #試試這行 : label_encoder.fit_transform(['a','a','b','c','a','b'])
        #cat_indices 為object的索引
        # Iterate through each column
        for i, col in enumerate(features):
            if features[col].dtype == 'object':
                # Map the categorical features to integers
                features[col] = label_encoder.fit_transform(np.array(features[col].astype(str)).reshape((-1,)))
                test_features[col] = label_encoder.transform(np.array(test_features[col].astype(str)).reshape((-1,)))

                # Record the categorical indices
                cat_indices.append(i)
    
    # Catch error if label encoding scheme is not valid
    #encoding參數給錯就報錯
    else:
        raise ValueError("Encoding must be either 'ohe' or 'le'")
        
    print('Training Data Shape: ', features.shape)
    print('Testing Data Shape: ', test_features.shape)
    
    # Extract feature names
    #取出特徵名稱
    feature_names = list(features.columns)
    
    #dataframe 轉成np.array
    # Convert to np arrays
    features = np.array(features)
    test_features = np.array(test_features)
    
    # Create the kfold object
    k_fold = KFold(n_splits = n_folds, shuffle = True, random_state = 50)
    
    # Empty array for feature importances
    #建立長度和特徵名稱一樣長且全為 0 的 np.array
    feature_importance_values = np.zeros(len(feature_names))
    
    
    # Empty array for test predictions
    #建立長度和測試資料筆數一樣長且全為 0 的 np.array
    test_predictions = np.zeros(test_features.shape[0])
    
    # Empty array for out of fold validation predictions
    #建立長度和訓練資料筆數一樣長且全為 0 的 np.array
    out_of_fold = np.zeros(features.shape[0])
    
    # Lists for recording validation and training scores
    #記錄驗證和訓練的成績
    valid_scores = []
    train_scores = []
    
    # Iterate through each fold
    # 迭代所有交叉驗證集
    for train_indices, valid_indices in k_fold.split(features):
        
        # Training data for the fold
        #交叉訓練資料
        train_features, train_labels = features[train_indices], labels[train_indices]
        
        # # Validation data for the fold
        # #交叉驗證資料
        valid_features, valid_labels = features[valid_indices], labels[valid_indices]
        
        # Create the model
        #建立 LGBM模型參數
        model = lgb.LGBMClassifier(n_estimators=10000, objective = 'binary', 
                                   class_weight = 'balanced', learning_rate = 0.05, 
                                   reg_alpha = 0.1, reg_lambda = 0.1, 
                                   subsample = 0.8, n_jobs = -1, random_state = 50)
        #訓練開始
        # Train the model
        model.fit(train_features, train_labels, eval_metric = 'auc',
                  eval_set = [(valid_features, valid_labels), (train_features, train_labels)],
                  eval_names = ['valid', 'train'], categorical_feature = cat_indices,
                  early_stopping_rounds = 100, verbose = 200)
        
        # Record the best iteration
        best_iteration = model.best_iteration_
        
        # Record the feature importances
        feature_importance_values += model.feature_importances_ / k_fold.n_splits
        
        # Make predictions
        test_predictions += model.predict_proba(test_features, num_iteration = best_iteration)[:, 1] / k_fold.n_splits
        
        # Record the out of fold predictions
        out_of_fold[valid_indices] = model.predict_proba(valid_features, num_iteration = best_iteration)[:, 1]
        
        # Record the best score
        valid_score = model.best_score_['valid']['auc']
        train_score = model.best_score_['train']['auc']
        
        valid_scores.append(valid_score)
        train_scores.append(train_score)
        
        # Clean up memory
        gc.enable()
        del model, train_features, valid_features
        gc.collect()
        
    # Make the submission dataframe
    submission = pd.DataFrame({'SK_ID_CURR': test_ids, 'TARGET': test_predictions})
    
    # Make the feature importance dataframe
    feature_importances = pd.DataFrame({'feature': feature_names, 'importance': feature_importance_values})
    
    # Overall validation score
    valid_auc = roc_auc_score(labels, out_of_fold)
    
    # Add the overall scores to the metrics
    valid_scores.append(valid_auc)
    train_scores.append(np.mean(train_scores))
    
    # Needed for creating dataframe of validation scores
    fold_names = list(range(n_folds))
    fold_names.append('overall')
    
    # Dataframe of validation scores
    metrics = pd.DataFrame({'fold': fold_names,
                            'train': train_scores,
                            'valid': valid_scores}) 
    
    return submission, feature_importances, metrics

In [34]:
model(X_train,X_test)

Training Data Shape:  (61502, 482)
Testing Data Shape:  (246009, 482)


KeyError: '[0, 2, 3, 5, 8, 9, 14, 15, 16, 17, 18, 19, 22, 23, 24, 25, 26, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 45, 47, 48, 49, 51, 52, 53, 55, 56, 57, 59, 61, 62, 63, 64, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 78, 79, 80, 81, 83, 85, 88, 89, 93, 94, 95, 97, 99, 102, 103, 105, 112, 113, 115, 116, 117, 121, 124, 126, 127, 128, 130, 131, 133, 135, 136, 137, 138, 139, 141, 143, 145, 147, 148, 149, 151, 152, 153, 154, 155, 156, 157, 159, 160, 161, 162, 164, 166, 167, 168, 169, 170, 174, 176, 177, 179, 180, 183, 184, 185, 187, 188, 189, 190, 191, 192, 194, 196, 198, 199, 200, 202, 203, 204, 206, 207, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 223, 225, 230, 234, 235, 236, 237, 238, 241, 243, 244, 245, 247, 248, 249, 250, 251, 252, 255, 258, 259, 261, 262, 265, 266, 267, 269, 270, 271, 273, 275, 276, 278, 279, 280, 281, 282, 283, 285, 286, 287, 288, 290, 295, 297, 300, 303, 305, 306, 307, 309, 314, 315, 320, 321, 322, 323, 326, 327, 328, 330, 331, 332, 333, 335, 338, 3

In [4]:
# from sklearn.model_selection import KFold
# from sklearn.metrics import roc_auc_score
# import lightgbm as lgb
# import gc

# def model(features, test_features, encoding = 'ohe', n_folds = 5):
    
#     """Train and test a light gradient boosting model using
#     cross validation. 
    
#     Parameters
#     --------
#         features (pd.DataFrame): 
#             dataframe of training features to use 
#             for training a model. Must include the TARGET column.
#         test_features (pd.DataFrame): 
#             dataframe of testing features to use
#             for making predictions with the model. 
#         encoding (str, default = 'ohe'): 
#             method for encoding categorical variables. Either 'ohe' for one-hot encoding or 'le' for integer label encoding
#             n_folds (int, default = 5): number of folds to use for cross validation
        
#     Return
#     --------
#         submission (pd.DataFrame): 
#             dataframe with `SK_ID_CURR` and `TARGET` probabilities
#             predicted by the model.
#         feature_importances (pd.DataFrame): 
#             dataframe with the feature importances from the model.
#         valid_metrics (pd.DataFrame): 
#             dataframe with training and validation metrics (ROC AUC) for each fold and overall.
        
#     """
    
#     #ID 取出
#     # Extract the ids
#     train_ids = features['SK_ID_CURR']
#     test_ids = test_features['SK_ID_CURR']
    
#     #目標取出
#     # Extract the labels for training
#     labels = features['TARGET']
    
#     #刪除train 的目標和ID ，刪除test的ID
#     # Remove the ids and target
#     features = features.drop(columns = ['SK_ID_CURR', 'TARGET'])
#     test_features = test_features.drop(columns = ['SK_ID_CURR'])
    
    
#     #分兩種方法'ohe':將train和test都做one hot建立虛擬變數(攤平)，並且根據兩者都有得col項目合併
#     # One Hot Encoding
#     if encoding == 'ohe':
#         features = pd.get_dummies(features)
#         test_features = pd.get_dummies(test_features)
        
#         # Align the dataframes by the columns
#         features, test_features = features.align(test_features, join = 'inner', axis = 1)
        
#         # No categorical indices to record
#         cat_indices = 'auto'
    
    
    
#     # Integer label encoding
#     elif encoding == 'le':
        
#         # Create a label encoder
#         label_encoder = LabelEncoder()
        
#         # List for storing categorical indices
#         cat_indices = []
        
#         #enumerate() 將資料變成((0,apptrain.columns[0]), (1,apptrain_columns[1])....)一對一放進 i 和 col跑迴圈
#         #LabelEncoder.fit_transform 將 object變數做重新編碼從 0 開始
#         #試試這行 : label_encoder.fit_transform(['a','a','b','c','a','b'])
#         #cat_indices 為object的索引
#         # Iterate through each column
#         for i, col in enumerate(features):
#             if features[col].dtype == 'object':
#                 # Map the categorical features to integers
#                 features[col] = label_encoder.fit_transform(np.array(features[col].astype(str)).reshape((-1,)))
#                 test_features[col] = label_encoder.transform(np.array(test_features[col].astype(str)).reshape((-1,)))

#                 # Record the categorical indices
#                 cat_indices.append(i)
    
#     # Catch error if label encoding scheme is not valid
#     #encoding參數給錯就報錯
#     else:
#         raise ValueError("Encoding must be either 'ohe' or 'le'")
        
#     print('Training Data Shape: ', features.shape)
#     print('Testing Data Shape: ', test_features.shape)
    
#     # Extract feature names
#     #取出特徵名稱
#     feature_names = list(features.columns)
    
#     #dataframe 轉成np.array
#     # Convert to np arrays
#     features = np.array(features)
#     test_features = np.array(test_features)
    
#     # Create the kfold object
#     k_fold = KFold(n_splits = n_folds, shuffle = True, random_state = 50)
    
#     # Empty array for feature importances
#     #建立長度和特徵名稱一樣長且全為 0 的 np.array
#     feature_importance_values = np.zeros(len(feature_names))
    
    
#     # Empty array for test predictions
#     #建立長度和測試資料筆數一樣長且全為 0 的 np.array
#     test_predictions = np.zeros(test_features.shape[0])
    
#     # Empty array for out of fold validation predictions
#     #建立長度和訓練資料筆數一樣長且全為 0 的 np.array
#     out_of_fold = np.zeros(features.shape[0])
    
#     # Lists for recording validation and training scores
#     #記錄驗證和訓練的成績
#     valid_scores = []
#     train_scores = []
    
#     # Iterate through each fold
#     # 迭代所有交叉驗證集
#     for train_indices, valid_indices in k_fold.split(features):
        
#         # Training data for the fold
#         #交叉訓練資料
#         train_features, train_labels = features[train_indices], labels[train_indices]
        
#         # Validation data for the fold
#         #交叉驗證資料
#         valid_features, valid_labels = features[valid_indices], labels[valid_indices]
        
#         # Create the model
#         #建立 LGBM模型參數
#         model = lgb.LGBMClassifier(n_estimators=10000, objective = 'binary', 
#                                    class_weight = 'balanced', learning_rate = 0.05, 
#                                    reg_alpha = 0.1, reg_lambda = 0.1, 
#                                    subsample = 0.8, n_jobs = -1, random_state = 50)
#         #訓練開始
#         # Train the model
#         model.fit(train_features, train_labels, eval_metric = 'auc',
#                   eval_set = [(valid_features, valid_labels), (train_features, train_labels)],
#                   eval_names = ['valid', 'train'], categorical_feature = cat_indices,
#                   early_stopping_rounds = 100, verbose = 200)
        
#         # Record the best iteration
#         best_iteration = model.best_iteration_
        
#         # Record the feature importances
#         feature_importance_values += model.feature_importances_ / k_fold.n_splits
        
#         # Make predictions
#         test_predictions += model.predict_proba(test_features, num_iteration = best_iteration)[:, 1] / k_fold.n_splits
        
#         # Record the out of fold predictions
#         out_of_fold[valid_indices] = model.predict_proba(valid_features, num_iteration = best_iteration)[:, 1]
        
#         # Record the best score
#         valid_score = model.best_score_['valid']['auc']
#         train_score = model.best_score_['train']['auc']
        
#         valid_scores.append(valid_score)
#         train_scores.append(train_score)
        
#         # Clean up memory
#         gc.enable()
#         del model, train_features, valid_features
#         gc.collect()
        
#     # Make the submission dataframe
#     submission = pd.DataFrame({'SK_ID_CURR': test_ids, 'TARGET': test_predictions})
    
#     # Make the feature importance dataframe
#     feature_importances = pd.DataFrame({'feature': feature_names, 'importance': feature_importance_values})
    
#     # Overall validation score
#     valid_auc = roc_auc_score(labels, out_of_fold)
    
#     # Add the overall scores to the metrics
#     valid_scores.append(valid_auc)
#     train_scores.append(np.mean(train_scores))
    
#     # Needed for creating dataframe of validation scores
#     fold_names = list(range(n_folds))
#     fold_names.append('overall')
    
#     # Dataframe of validation scores
#     metrics = pd.DataFrame({'fold': fold_names,
#                             'train': train_scores,
#                             'valid': valid_scores}) 
    
#     return submission, feature_importances, metrics